#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

- Source Files has the Structured data with comma seperator (CSV) and Semistructured data (json)
- Header, No comments is there in the data
-Total columns are 5 in once source1 csv and 7 in the source2 csv.
- Total columns in json are 11
- Data Quality 
- - Null columns and null records are there
- - duplicate rows & Duplicate id keys
- - format issues are there
- - Schema evolution is there
- - Number of columns are more or less than the expected
- - Mal data is found in the field value

####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
#Apply inferSchema and toDF to create a DF and analyse the actual data.
df_source1 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source1")
df_source2 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source2")
#Analyse the schema, datatypes, columns etc.,
#For Source file 1 we have the following schema
"""shipment_id:string
first_name:string
last_name:string
age:string
role:string"""""
#For Source file 2 we have the following schema , which has 3 new fields added(role,hub_location,vehicle_type)
"""shipment_id:string
first_name:string
last_name:string
age:string
role:string
hub_location:string
vehicle_type:string"""
print(df_source1.columns)
print(df_source2.columns)
print(df_source1.dtypes)
print(df_source2.dtypes)
#Analyse the duplicate records count and summary of the dataframe.
total_count_s1=df_source1.count()
print(f"The total count of records in source file 1 is {total_count_s1}")
total_count_s2=df_source2.count()
print(f"The total count of records in source file 2 is {total_count_s2}")
print(total_count_s2)
distinct_count_s1=df_source1.distinct().count()
print(f"The total distinct count of records in source file 1 is {distinct_count_s1}")
distinct_count_s2=df_source2.distinct().count()
print(f"The total distinct count of records in source file 2 is {distinct_count_s2}")
df_source1.describe().show()
df_source2.describe().show()


###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:
3. fewer columns than expected
4. more columns than expected

In [0]:
#Create a Spark Session Object
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark=SparkSession.builder.appName("Logistics-Data").getOrCreate()

#Shipment IDs that appear in both master_v1 and master_v2
print("Shipment IDs that appear in both master_v1 and master_v2")
common_shipments = df_source1.select("shipment_id") \
    .intersect(df_source1.select("shipment_id"))
common_shipments.show()
#shipment_id is non-numeric
non_numeric_shipment = df_source1.filter(
    ~col("shipment_id").rlike("^[0-9]+$")
)

non_numeric_shipment.show()

#age is not an integer
invalid_age = df_source1.filter(
    ~col("age").rlike("^[0-9]+$")
)

invalid_age.show()


###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:

from pyspark.sql.functions import lit, col,expr

#Read both sources without enforcing schema
df_source1 = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source1")
)
df_source2 = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source2")
)

#Align both to canonical schema
df_system1 = (
    df_source1
    .select(
        col("shipment_id"),
        col("first_name"),
        col("last_name"),
        expr("try_cast(age as int)").alias("age"),
        col("role")
    )
    .withColumn("data_source", lit("system1"))
)

df_system2 = (
    df_source2
    .select(
        col("shipment_id"),
        col("first_name"),
        col("last_name"),
        expr("try_cast(age as int)").alias("age"),
        col("role"),
        col("hub_location"),
        col("vehicle_type")
    )
    .withColumn("data_source", lit("system2"))
)

df_canonical = df_system1.unionByName(df_system2, allowMissingColumns=True)
df_canonical.printSchema()
df_canonical.show(5)


#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
from pyspark.sql.functions import *
#Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role
df_clean_1 = df_canonical.dropna(subset=["shipment_id", "role"])
df_clean_1.show(10)
#Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name
df_clean_2 = df_clean_1.filter(
    ~(col("first_name").isNull() & col("last_name").isNull())
)
df_clean_2.show(10)

#Join Readiness Rule - Drop records where the join key is null: shipment_id
df_clean = df_clean_2.filter(col("shipment_id").isNotNull())
print(df_clean.count())
df_check=df_clean.filter(col('shipment_id').isNull())
print(df_check.count())

#4. Age Defaulting Rule - Fill NULL values in the age column with: -1
df_scrub_1 = df_clean.na.fill({"age": -1})
df_scrub_1.show(10)     
#5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN
df_scrub_2 = df_scrub_1.fillna({"vehicle_type": "UNKNOWN"})
#6. Invalid Age Replacement - Replace the following values in age: "ten" to -1 "" to -1
df_scrub_3 = df_scrub_2.withColumn( "age",when((col("age") == "ten") | (col("age") == ""), -1))
    #.otherwise(col("age"))
#)
df_scrub_3  = df_scrub_2.withColumn(
    "age",
    when(col("age").rlike("^[0-9]+$"), col("age").cast("int")).otherwise(-1)
)
#7. Vehicle Type Normalization - Replace inconsistent vehicle types: truck to LMV bike to TwoWheeler
df_final = df_scrub_3.replace({
        "truck": "LMV",
        "bike": "TwoWheeler"
    },
    subset=["vehicle_type"]
)
df_final.show(10)   

####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

Detail Dataframe creation <br>
1. Read Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

In [0]:
from pyspark.sql.functions import *
df_shipment_detail_json = (
    spark.read
    .option("multiline", "true")   # use only if JSON is multiline
    .json("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_shipment_detail_3000.json")
)

df_shipment_detail_json.show(10)
df_shipment_detail_json.printSchema()


Standardizations:<br>

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: DF of merged(logistics_source1 & logistics_source2) <br>
age: Cast String to Integer<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>

In [0]:
from pyspark.sql.types import DecimalType
#1. Add a column<br> 
df_detail_std = (
    df_shipment_detail_json
    .withColumn("domain", lit("Logistics"))
    .withColumn(
        "shipment_date",
        coalesce(
            try_to_date(col("shipment_date"), "yyyy-MM-dd"),
            try_to_date(col("shipment_date"), "yy-MM-dd")
        )
    )
    .withColumn("vehicle_type", upper("vehicle_type"))
    .withColumn("source_city", initcap("source_city"))
)
#2. Column Uniformity: 
#csv file (Source1 and 2))
df_master_std = df_final.withColumn(
    "role",
    lower(col("role"))
).withColumn(
    "vehicle_type",
    upper(col("vehicle_type"))
)
#Json file 


df_detail_std.show(10)
#3. Format Standardization:
df_detail_std_nw = df_detail_std.withColumn(
    "shipment_date",
    to_date(col("shipment_date"))
)
df_detail_std = df_detail_std.withColumn(
    "shipment_cost",
    round(col("shipment_cost"), 2).cast(DecimalType(10, 2))
)
#df_detail_std_nw.show(10)
df_detail_std_nw.select("shipment_date").show(truncate=False)
df_detail_std_nw.printSchema()

df_master_std.printSchema()
df_detail_std.printSchema()



4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.
Source File: DF of merged(logistics_source1 & logistics_source2)
age: Cast String to Integer
Source File: DF of logistics_shipment_detail_3000.json
shipment_weight_kg: Cast to Double
Source File: DF of logistics_shipment_detail_3000.json
is_expedited: Cast to Boolean

In [0]:
#Standardizing column data types to fix schema drift and enable mathematical operations
df_master_std = df_master_std.withColumn(
    "age",
    col("age").cast("int")
)
df_master_std.printSchema()
df_detail_std = df_detail_std.withColumn(
    "shipment_weight_kg",
    col("shipment_weight_kg").cast("double")
)
#is_expedited: Cast to Boolean<br> is not available in source file hence not used here
df_detail_std.printSchema()


5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>

In [0]:
df_master_std = (
    df_master_std
    .withColumnRenamed("first_name", "staff_first_name")
    .withColumnRenamed("last_name", "staff_last_name")
    .withColumnRenamed("hub_location", "origin_hub_city")
)
df_master_std.printSchema()

6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
df_master_sel = df_master_std.select(
    "shipment_id",
    "staff_first_name",
    "staff_last_name",
    "role",
    "origin_hub_city"
)

df_detail_sel = df_detail_std.select(
    "shipment_id",
    "shipment_cost"
)
df_final = (
    df_master_sel
    .join(df_detail_sel, on="shipment_id", how="left")
)
df_final = df_final.withColumn(
    "ingestion_timestamp",
    current_timestamp()
)
df_final = df_final.select(
    "shipment_id",
    "staff_first_name",
    "staff_last_name",
    "role",
    "origin_hub_city",
    "shipment_cost",
    "ingestion_timestamp"
)
df_final.printSchema()
df_final.show(truncate=False)

Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

In [0]:
#Record level duplicate check
#df_record_dedup = df_final.dropDuplicates()
#df_record_dedup.count()
#Column level duplicate check
df_col_dedup = df_final.dropDuplicates(["shipment_id"])
df_col_dedup.count()

##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

**1. Add Audit Timestamp (`load_dt`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

In [0]:
#Add Audit Timestamp (load_dt) Source File: logistics_source1 and logistics_source2
#Create Full Name (full_name) Source File: logistics_source1 and logistics_source2
df_source1 = df_source1.withColumn(
    "full_name",
    concat_ws(" ", col("first_name"), col("last_name"))
)
df_source2 = df_source2.withColumn(
    "full_name",
    concat_ws(" ", col("first_name"), col("last_name"))
)
df_source1.select("first_name", "last_name", "full_name").show(3)
df_source2.select("first_name", "last_name", "full_name").show(3)
df_source1_audit = df_source1.withColumn(
    "load_dt",
    current_timestamp()
)
df_source2_audit = df_source2.withColumn(
    "load_dt",
    current_timestamp()
)
df_source1_audit.printSchema()
df_source2_audit.printSchema()



**3. Define Route Segment (`route_segment`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

In [0]:
#Define Route Segment (route_segment) 
df_detail_std = df_detail_std.withColumn(
    "route_segment",
    concat_ws("-", col("source_city"), col("destination_city"))
)
df_detail_std.select("route_segment").show(5)

df_detail_std = df_detail_std.withColumn(
    "vehicle_identifier",
    concat_ws("_", col("vehicle_type"), col("shipment_id"))
)
df_detail_std.select(col('vehicle_identifier')).show(5)

###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

In [0]:
#Derive shipment_year
df_detail_std=df_detail_std.withColumn("shipment_year",year(col("shipment_date")))
df_detail_std.select("shipment_date","shipment_year").show(5)

#Derive shipment_month
df_detail_std=df_detail_std.withColumn("shipment_month",month(col("shipment_date")))
df_detail_std.select("shipment_date","shipment_month").show(5)

#Flag Weekend Operations (is_weekend)
df_detail_std = df_detail_std.withColumn(
    "is_weekend",
    when(dayofweek(col("shipment_date")).isin(1, 7), True)
    .otherwise(False)
)
df_detail_std .select ("shipment_date","shipment_year","shipment_month","is_weekend").show(5)
df_detail_std .select ("shipment_date","shipment_year","shipment_month","is_weekend").filter(col("is_weekend")==True).show(5)

###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

In [0]:
#1. Calculate Unit Cost (cost_per_kg)
df_detail_std = df_detail_std.withColumn(
    "cost_per_kg",
    when(
        (col("shipment_weight_kg").isNotNull()) & (col("shipment_weight_kg") > 0),
        round(col("shipment_cost") / col("shipment_weight_kg"), 2)
    ).otherwise(None)
)
#2. Track Shipment Age (days_since_shipment)
df_detail_std = df_detail_std.withColumn(
    "days_since_shipment",
    datediff(current_date(), col("shipment_date"))
)
#3. Compute Tax Liability (tax_amount)
df_detail_std = df_detail_std.withColumn(
    "tax_amount",
    round(col("shipment_cost") * 0.18, 2)
)

df_detail_std.select(
    "shipment_cost",
    "shipment_weight_kg",
    "cost_per_kg",
    "days_since_shipment",
    "tax_amount"
).show(5, truncate=False)
#Add Audit Timestamp (load_dt)

###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

In [0]:
#1. Remove Redundant Name Columns
df_source1 = df_source1.drop("first_name", "last_name")
df_source2 = df_source2.drop("first_name", "last_name")
df_source1.printSchema()
df_source2.printSchema()

##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

In [0]:
#1. Splitting (Extraction) 
df_detail_std = df_detail_std.withColumn(
    "order_prefix",
    regexp_extract(col("order_id"), "^[A-Za-z]+", 0)
).withColumn(
    "order_sequence",
    regexp_extract(col("order_id"), "[0-9]+$", 0)
)
df_detail_std = df_detail_std.withColumn(
    "ship_year",
    year(col("shipment_date"))
).withColumn(
    "ship_month",
    month(col("shipment_date"))
).withColumn(
    "ship_day",
    dayofmonth(col("shipment_date"))
)
df_detail_std.select("order_id","order_prefix","order_sequence","ship_year","ship_month","ship_day").show(5)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

In [0]:
#2. Merging (Concatenation)
df_detail_std = df_detail_std.withColumn(
    "route_lane",
    concat_ws("->", col("source_city"), col("destination_city"))
)
df_detail_std .select( "order_id",
    "order_prefix",
    "order_sequence",
    "shipment_date",
    "ship_year",
    "ship_month",
    "ship_day",
    "route_lane"
).show(5,truncate=False)

## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

#### **UDF1: Complex Incentive Calculation**

In [0]:
#Python function for calculate bonus:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

def calculate_bonus(role, age, salary):
    if role == "driver" and age is not None and age >= 0:
        if age > 50:
            return salary * 0.15
        elif age < 30:
            return salary * 0.05
    return 0.0
    
#converting the function to udf :
calculate_bonus_udf = udf(calculate_bonus, DoubleType())
df_canonical.select(
    col("role"),
    lower(trim(col("role"))).alias("clean_role"),
    col("age")
).distinct().show(truncate=False)

df_with_bonus = df_canonical.withColumn(
    "projected_bonus",
    calculate_bonus_udf(
        lower(col("role")),
        col("age"),
        lit(15000)
    )
)
#Validate
df_with_bonus.select(
    "role",
    "age",
    "projected_bonus"
).where(col("role") == "driver") \
 .show(truncate=False)

#### **UDF2: PII Masking (Privacy Compliance)

In [0]:
df_masked = df_source1.withColumn(
    "masked_name",
    concat(
        substring(col("full_name"), 1, 2),       # First 2 chars
        lit("****"),                              # Mask
        substring(col("full_name"), length(col("full_name")), 1)  # Last char
    )
)

df_masked1 = df_source1.withColumn(
    "masked_name",
    when(
        length(col("full_name")) >= 3,
        concat(
            substring(col("full_name"), 1, 2),
            lit("****"),
            substring(col("full_name"), length(col("full_name")), 1)
        )
    ).otherwise(col("full_name"))
)

df_masked.show(5,truncate=False)

## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `priority_flag` (Descending).

**7. Limit (Top-N Analysis)**<br>
Source File: json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

#### **1. Select (Projection)** <br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.



In [0]:
#Select (Projection)

df_projection = df_canonical.select(
    col("first_name"),
    col("role"),
    col("hub_location")
).filter(col("hub_location").isNotNull())
df_projection.show(5,truncate=False)
df_projection.printSchema()



#### **2. Filter (Selection)**<br>
Source File: json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

In [0]:
df_operational_issues = df_detail_std.filter(
    col("shipment_status").isin("DELAYED", "RETURNED")
)
df_operational_issues.show(truncate=False)
df_senior_staff = df_canonical.filter(col("age") > 50)
df_senior_staff.select(
    "shipment_id",
    "first_name",
    "last_name",
    "age",
    "role"
).show(truncate=False)

#### **3. Derive Flags & Columns (Business Logic)**<br>
Source File: json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

In [0]:
df_detail_flagged = df_detail_std.withColumn(
    "is_high_value",
    when(col("shipment_cost") > 50000, True).otherwise(False)
)
df_detail_flagged = df_detail_flagged.withColumn(
    "is_weekend",
    when(dayofweek(col("shipment_date")).isin(1, 7), True).otherwise(False)
)
df_detail_flagged.select(
    "shipment_id",
    "shipment_date",
    "shipment_cost",
    "is_high_value",
    "is_weekend"
).show(truncate=False)

#### **4. Format (Standardization)**<br>
Source File: json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

In [0]:
df_detail_fmt = df_detail_std.withColumn(
    "shipment_cost_fmt",
    concat(lit("₹"), format_number("shipment_cost", 2))
)
df_detail_fmt = df_detail_fmt.withColumn(
    "source_city",
    upper("source_city")
)
df_detail_fmt.select(
    "shipment_id",
    "shipment_cost",
    "shipment_cost_fmt",
    "source_city"
).show(truncate=False)

#### **5. Group & Aggregate (Summarization)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

In [0]:
#Aggreagtion
df_staff_by_hub = (
    df_canonical
    .groupBy("hub_location")
    .agg(count("*").alias("staff_count"))
)

df_staff_by_hub.show(5,truncate=False)
df_staff_by_hub.printSchema()

#### **6. Sorting (Ordering)**<br>
Source File: json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `shipment_cost` (Descending).

In [0]:
df_sorted_by_cost = df_detail_std.orderBy(
    col("shipment_cost").desc()
)
df_sorted_by_cost.select(
    "shipment_id",
    "shipment_cost"
).show(5,truncate=False)

df_dispatch_schedule = df_detail_std.orderBy(
    col("shipment_date").asc(),
    col("shipment_cost").desc()
)
df_dispatch_schedule.select(
    "shipment_id",
    "shipment_date"    
).show(5,truncate=False)

#### **7. Limit (Top-N Analysis)**<br>
Source File: json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

In [0]:
df_top_delayed = (
    df_detail_std
    .filter(col("shipment_status") == "DELAYED")
    .orderBy(col("shipment_cost").desc())
    .limit(10)
)
df_top_delayed.select(
    "shipment_id",
    "shipment_status",
    "shipment_cost",
    "shipment_date"
).show(truncate=False)

## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

### **3. Lookup & Enrichment**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.


#### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> logistics_shipment_detail_3000.json<br>
##### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.


In [0]:
df_master_clean = (
    df_master_std
    .filter(col("shipment_id").rlike("^[0-9]+$"))
    .withColumn("shipment_id", col("shipment_id").cast("string"))
)
df_master_clean .printSchema()
df_detail_clean = (
    df_detail_std
    .filter(col("shipment_id").rlike("^[0-9]+$"))
    .withColumn("shipment_id", col("shipment_id").cast("string"))
)
df_staff_shipments_inner = (
    df_master_clean.alias("s")
    .join(
        df_detail_clean.alias("d"),
        on=col("s.shipment_id") == col("d.shipment_id"),
        how="inner"
    )
)
df_staff_shipments_inner.select(
    "s.shipment_id",
    "s.staff_first_name",
	"s.staff_last_name",   
    "s.role",
    "d.shipment_status",
    "d.shipment_cost"
).show(5,truncate=False)

#B. Left Join – Idle Resource Check
df_staff_shipments_left = (
    df_master_clean.alias("s")
    .join(
        df_detail_clean.alias("d"),
        on=col("s.shipment_id") == col("d.shipment_id"),
        how="left"
    )
)
#Filter Idle Staff
df_idle_staff = df_staff_shipments_left.filter(
    col("d.shipment_id").isNull()
)

df_idle_staff.select(
    "s.shipment_id",
    "s.staff_first_name",
	"s.staff_last_name", 
    "s.role",
    "s.origin_hub_city"
).show(5,truncate=False)

##### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.


In [0]:
#2 Infrequent Simple Joins
staff_a = df_master_clean.alias("a")
staff_b = df_master_clean.alias("b")
df_peer_staff = (
    staff_a
    .join(
        staff_b,
        col("a.origin_hub_city") == col("b.origin_hub_city"),
        "inner"
    )
    .filter(col("a.shipment_id") != col("b.shipment_id"))
)
df_peer_staff.select(
    col("a.staff_first_name").alias("staff_A"),
    col("b.staff_first_name").alias("staff_B"),
    col("a.origin_hub_city")
).show(5,truncate=False)

##### **1.3 Advanced Joins (Semi and Anti)**
**Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

In [0]:
drivers_with_shipments = (
  df_master_clean
    .join(df_detail_clean, on="shipment_id", how="left_semi")
)
drivers_with_shipments.show(5)

drivers_without_shipments = (
    df_master_clean
    .join(df_detail_clean, on="shipment_id", how="left_anti")
)
drivers_without_shipments.show(5)   

#### **2. Lookup**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

In [0]:
df_city_master = spark.read.format("csv") \
    .option("header", "true") \
    .load("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/Master_City_List.csv")

staff_df_clean = df_source1.unionByName(df_source2,allowMissingColumns=True)

df_city_master = df_city_master.withColumn(
    "city_name",
    upper(trim(col("city_name")))
)
valid_hubs_df = staff_df_clean .join(
    df_city_master,
    staff_df_clean.hub_location == df_city_master.city_name,
    how="left_semi"
)
valid_hubs_df.show(2,truncate=False)
invalid_hubs_df = staff_df_clean .join(
    df_city_master,
    staff_df_clean .hub_location == df_city_master.city_name,
    how="left_anti"
)
invalid_hubs_df.show(2,truncate=False)
#staff_df_clean.filter(col("hub_location").isNotNull()).show(2,truncate=False)
#df_city_master .show(2)

#### **3. Lookup & Enrichment**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.


In [0]:


df_city_master = df_city_master.withColumn(
    "city_name", upper(trim(col("city_name")))
)
staff_geo_df = (
    staff_df_clean
    .join(
        df_city_master,
        col("hub_location") == col("city_name"),
        how="left"
    )
)
staff_geo_df.filter(col("hub_location").isNotNull()).show(5)

In [0]:
#df_detail_std.printSchema()
#df_city_master.printSchema()
staff_df_clean .printSchema()

#### **4. Schema Modeling (Denormalization)**<br>
Source Files: All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

In [0]:
df_master_clean = (
    df_master_std
    .withColumn("shipment_id", col("shipment_id").cast("string"))
)

df_detail_clean = (
    df_detail_std
    .withColumn("shipment_id", col("shipment_id").cast("string"))
)
df_master_clean.printSchema()
df_detail_clean.printSchema()
#Denormalize (Correct Join Direction)
wide_shipment_history = (
    df_detail_clean.alias("d")
    .join(
        df_master_clean.alias("m"),
        col("d.shipment_id") == col("m.shipment_id"),
        how="left"
    )
)
#wide_shipment_history.show(5)
wide_shipment_history = wide_shipment_history.select(
    # Key
    "d.shipment_id",
    # Detail-level facts
    "m.staff_first_name",
    "m.staff_last_name",
	"m.age",
    "d.shipment_status",
    "d.cargo_type",

    # Master attributes
    "m.vehicle_type",
	"d.shipment_cost",
    "m.origin_hub_city",
    "d.source_city"
   ).show(5)

#### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, row_number
df_source2 = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source2")
)

df_shipment_detail_json = (
    spark.read
    .option("multiline", "true")
    .json("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_shipment_detail_3000.json")
)
df_source2_clean = (
    df_source2
    .withColumn("shipment_id", col("shipment_id").cast("string"))
   
)
df_shipment_detail_flat = (
    df_shipment_detail_json
    .withColumn("shipment_id", col("shipment_id").cast("string"))
    .select(
        col("shipment_id"),
        col("shipment_cost"),
        col("shipment_date")
    )
)
base_df = (
    df_shipment_detail_flat.alias("d")
    .join(
        df_source2_clean.alias("m"),
        col("d.shipment_id") == col("m.shipment_id"),
        "left"
    )
)

base_df.printSchema()

driver_cost_df = (
    base_df
    .groupBy("hub_location")
    .agg(
        sum("shipment_cost").alias("total_shipment_cost")
    )
)

hub_window = Window.partitionBy("hub_location") \
                   .orderBy(col("total_shipment_cost").desc())

ranked_df = (
    driver_cost_df
    .withColumn("dense_rank", dense_rank().over(hub_window))
    .withColumn("row_number", row_number().over(hub_window))
)
top_3_dense = ranked_df.filter(col("dense_rank") <= 3)
top_3_dense.show(10)


#### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

In [0]:
base_df = base_df.withColumn(
    "shipment_date",
    to_date(col("shipment_date"))
)

driver_window = Window.partitionBy("m.shipment_id") \
                      .orderBy(col("shipment_date"))
base_df.printSchema()
idle_time_df = (
    base_df
    .withColumn(
        "prev_shipment_date",
        lag("shipment_date").over(driver_window)
    )
    .withColumn(
        "idle_days",
        datediff(
            col("shipment_date"),
            col("prev_shipment_date")
        )
    )
)
idle_time_df.filter(col("idle_days").isNotNull()).show(5)

#### **7. Set Operations**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

In [0]:
df_source1 = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source1")
)
df_source2 = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv("/Volumes/ag_catalog/ag_practice/ag_volume/source_dir/logistics/logistics_source2")
)
staff_df = df_source1.unionByName(df_source2,allowMissingColumns=True)
staff_df.show(5)
#Identify IDs present in BOTH Source1 and Source2
common_staff_ids = (
    df_source1
    .select("shipment_id")
    .intersect(
        df_source2.select("shipment_id")
    )
)
common_staff_ids.show(5)
#Identify  IDs present in Source2 but missing from Source1
new_miss_src = (
    df_source2
    .select("shipment_id")
    .exceptAll(
        df_source1.select("shipment_id")
    )
)
new_miss_src.show(5)

#### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

In [0]:
from pyspark.sql.functions import *
base_df = (
    df_shipment_detail_flat.alias("d")
    .join(
        df_source2.alias("m"),
        col("d.shipment_id") == col("m.shipment_id"),
        "left"
    )
)
base_df.show(5)
rollup_df = (
    base_df
    .rollup("hub_location", "vehicle_type")
    .agg(
        sum("shipment_cost").alias("total_shipment_cost")
    )
)
rollup_df.show(5)
cube_df = (
    base_df
    .cube("hub_location", "vehicle_type")
    .agg(
        sum("shipment_cost").alias("total_shipment_cost")
    )
)
cube_df.show(5)
final_df = cube_df.withColumn(
    "aggregation_level",
    when(col("hub_location").isNull() & col("vehicle_type").isNull(), "GRAND_TOTAL")
    .when(col("vehicle_type").isNull(), "HUB_TOTAL")
    .when(col("hub_location").isNull(), "VEHICLE_TOTAL")
    .otherwise("HUB_VEHICLE_TOTAL")
)
final_df.show(5)

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.